## Senado
<p><code>Python en Jupyter Notebook</code></p>
<p>Creado por <code>Giancarlo Ortiz</code> para explicar el método electoral <code>no proporcional</code></p>

El sistema electoral vigente en Colombia se encuentra definido en la reforma constitucional del [acto legislativo 01 de 2003](http://www.secretariasenado.gov.co/senado/basedoc/acto_legislativo_01_2003.html) que define metodos de cifra repartidora y umbral para las elecciones de las dos Cámaras del congreso de la republica. 

## Agenda
1. Métodos no lineales
1. Cifra repartidora
1. Método d'Hondt
1. Umbral

In [7]:
import colorama

### 1. Métodos no lineales
---
En un sistema electoral existen métodos [no lineales](https://es.wikipedia.org/wiki/Sistema_no_lineal) para repartir los escaños de un cuerpo colegiado, los mas conocidos se basan en el promedio mayor y el resto mayor; los Métodos de promedio mayor son menos lineales y asignan mas escaños a los promedios más altos y los metodos de resto mayor asignan mas escaños a los retos más altos.

### 2. Cifra repartidora
---
Los métodos de promedio mayor se basan en la division sucesiva de los votos de cada una de las listas por una sucesion de números para producir una tabla de cocientes llamados promedios, finalmente de todos estos promedios se escogen los mayores de acuerdo a una cifra minima o cifra repartidora.

### 3. Método d'Hondt 
---
Los metodos de promedio mayor o de cifra repartidora comúnmente utilizados son el método d'Hondt y el Método Sainte-Laguë o Webster, en el primero los divisores son los números naturales (1, 2, 3, ...) y en el segundo solo los números impares (1, 3, 5, ...), de modo que las formulas de los cocientes d'Hondt y de Webster en cada iteración se pueden expresar como:

\begin{equation*}
Ch_{i,j} = \frac{V_i}{S_j + 1}
\end{equation*}

\begin{equation*}
Cw_{i,j} = \frac{V_i}{2S_j + 1}
\end{equation*}


><br>**Donde:** 
>* $Ch_j$ es el coeficiente d'Hondt.
>* $Cw_j$ es el coeficiente de Webster.
>* $V_i$ es el número total de votos para la i-ésima lista.
>* $S_j$ es el número j-ésimo de escaños conseguidos por la i-ésima lista.
><br><br>


### 4. Umbral 
---
En cualquiera de los métodos para repartir esca;os se puede establecer un umbral o limite inferior a partir del cual una lista no acumula esca;os sin importar la cifra repartidora.

\begin{equation*}
Umbral = \frac{V}{2c}
\end{equation*}

### <code>Ejemplo:</code> método D'Hondt
---
Cálculo de credenciales al senado para cada lista en el escrutinio final de las elecciones legislativas de 2018, realizadas el domingo 11 de marzo en Colombia.

In [8]:
# Funciones
def proporcional(votos: list, curules: int) -> list:
    ''' Funcion que Calcula las curules para cada lista proporcionalmente '''
    total_votos = sum(votos)
    curules_por_lista = []
    for votos_lista in votos:
        curules_por_lista.append(votos_lista*curules/total_votos)
    return curules_por_lista


def repartidora(votos: list, curules: int) -> int:
    ''' Funcion que Calcula la cifra repartidora '''
    values = []
    tabla = [[0 for col in range(curules)] for row in range(len(votos))]
    lista = -1
    for votos_lista in votos:
        lista += 1
        for item in range(curules):
            value = votos_lista/(item+1)
            values.append(value)
            tabla[lista][item] = value
    return round(sorted(values)[-curules-2])


def credenciales(votos: list, repartidora: int, umbral: int) -> list:
    ''' Funcion que Calcula las curules para cada lista con el metodo D'Hondt '''
    curules = []
    for item in votos:
        if item >= umbral:
            curules.append(int(item/repartidora))
        else:
            curules.append(0)
    return curules


In [9]:
# Datos votación escrutinio final 2018
votos_listas = [2_508_534, 2_152_793, 1_930_197, 1_896_283, 1_854_349, 1_312_291,
                725_244, 520_462, 495_106, 464_916, 345_342, 103_368, 58_023, 55_587, 34_555, 13_439]
votos_blancos = 842_286
curules_senado_ordinaria = 100
curules_senado_especial = 8


In [10]:
# Cálculo de credenciales senado escrutinio final 2018
votos_validos = sum(votos_listas) + votos_blancos
umbral = votos_validos*(3/100)
cifra_repartidora = repartidora(votos_listas, curules_senado_ordinaria)
curules_por_lista = credenciales(votos_listas, cifra_repartidora, umbral)


In [13]:
print(f"*"*30)
print(f"Validos      : {votos_validos:15,.0f}")
print(f"Umbral       : {umbral:15,.0f}")
print(f"Repartidora  : {cifra_repartidora:15,.0f}")
print(f"*"*37)
for item in range(len(votos_listas)):
    print(
        f"|{item+1:3.0f} |{votos_listas[item]:15,.0f} |{curules_por_lista[item]:5.0f} |{votos_listas[item]*curules_senado_ordinaria/sum(votos_listas):5.0f} |")
print(f"*"*37)
print(f"|  T |{sum(votos_listas):15,.0f} |{sum(curules_por_lista):5.0f} | {sum(curules_por_lista):5.0f} |")


******************************
Validos      :      15,312,775
Umbral       :         459,383
Repartidora  :         132,028
*************************************
|  1 |      2,508,534 |   19 |   17 |
|  2 |      2,152,793 |   16 |   15 |
|  3 |      1,930,197 |   14 |   13 |
|  4 |      1,896,283 |   14 |   13 |
|  5 |      1,854,349 |   14 |   13 |
|  6 |      1,312,291 |    9 |    9 |
|  7 |        725,244 |    5 |    5 |
|  8 |        520,462 |    3 |    4 |
|  9 |        495,106 |    3 |    3 |
| 10 |        464,916 |    3 |    3 |
| 11 |        345,342 |    0 |    2 |
| 12 |        103,368 |    0 |    1 |
| 13 |         58,023 |    0 |    0 |
| 14 |         55,587 |    0 |    0 |
| 15 |         34,555 |    0 |    0 |
| 16 |         13,439 |    0 |    0 |
*************************************
|  T |     14,470,489 |  100 |


---
## Mas Recursos

- [Métodos del promedio mayor](https://es.wikipedia.org/wiki/M%C3%A9todo_de_promedios_mayores) (Wikipedia)
- [Método d'Hondt](https://es.wikipedia.org/wiki/Sistema_D%27Hondt) (Wikipedia)
- [Método Webster](https://es.wikipedia.org/wiki/M%C3%A9todo_Sainte-Lagu%C3%AB) (Wikipedia)
- [Métodos del resto mayor](https://es.wikipedia.org/wiki/M%C3%A9todo_del_resto_mayor) (Wikipedia)